# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup - - - - - - - - - - -   - - - - - - - - - 3/15/2020 - Randy Dettmer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
# load output file generated in WeatherPy (CSV)

file = "Data/output_data.csv"
df1 = pd.read_csv(file)

#review file
df1.head()

,City,Country,Latitude,Longitude,Max Temp(F),Humidity(%),Cloudiness(%),Wind Speed(MPH),Date
0,Srednekolymsk,RU,67.45,153.68,-9.78,88.0,0.0,3.02,1.584312e+09
1,Severnoye,RU,56.35,78.36,33.04,86.0,100.0,21.41,1.584312e+09
2,Port Alfred,ZA,-33.59,26.89,63.00,97.0,100.0,3.00,1.584312e+09
3,Cape Town,ZA,-33.93,18.42,64.99,63.0,20.0,20.80,1.584312e+09
4,Bud,NO,39.45,-86.18,46.00,83.0,90.0,9.17,1.584312e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [10]:
#generate coordinates for heat map
locations = df1[["Latitude","Longitude"]]
humidity = df1["Humidity(%)"]

In [14]:
#plot heat map - locations vs humidity
fig = gmaps.figure()

#create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=True, max_intensity=5, 
                                 point_radius=2)
#add layer
fig.add_layer(heat_layer)

#display figure
fig

Figure(layout=FigureLayout(height='420px'))

The heat map for all locations show that all locations are randomly distributed across the globe.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
#narrow down data frame for specific weather conditions
#Max Temp(F) between 70 and 80 degrees
#Wind Speed(MPH) less than 10 mph
#Cloudiness(%) is zero
df2 = df1
df2 = df2.loc[(df2["Max Temp(F)"] <=80) & (df2["Max Temp(F)"] >=70)]
df2 = df2.loc[(df2["Wind Speed(MPH)"] <10)]
df2 = df2.loc[(df2["Cloudiness(%)"] ==0)]
df2.head(20)

,City,Country,Latitude,Longitude,Max Temp(F),Humidity(%),Cloudiness(%),Wind Speed(MPH),Date
11,Taoudenni,ML,22.68,-3.98,73.33,10.0,0.0,9.35,1.584312e+09
94,Port Hedland,AU,-20.32,118.57,78.80,88.0,0.0,4.70,1.584312e+09
159,Srivardhan,IN,18.03,73.02,75.43,45.0,0.0,5.44,1.584312e+09
201,Marsh Harbour,BS,26.54,-77.06,72.72,70.0,0.0,9.22,1.584312e+09
316,Kibala,AO,9.11,18.35,76.78,14.0,0.0,4.70,1.584313e+09
324,Otjimbingwe,NaN,-22.35,16.13,72.36,36.0,0.0,1.83,1.584313e+09
372,Berbera,SO,10.44,45.01,77.94,82.0,0.0,5.68,1.584313e+09
374,Yulara,AU,-25.24,130.99,71.60,18.0,0.0,6.93,1.584312e+09
379,Bethanien,NaN,-26.50,17.16,73.62,29.0,0.0,4.54,1.584313e+09
382,Najran,SA,17.49,44.13,71.60,35.0,0.0,4.70,1.584313e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
#create hotel data frame based off of above weather criteria
hotel_df = pd.DataFrame(df2, columns=["City", "Country", "Latitude", "Longitude"])
#hotel_df["Hotel Name"] = np.nan

hotel_df.head(20)

,City,Country,Latitude,Longitude
11,Taoudenni,ML,22.68,-3.98
94,Port Hedland,AU,-20.32,118.57
159,Srivardhan,IN,18.03,73.02
201,Marsh Harbour,BS,26.54,-77.06
316,Kibala,AO,9.11,18.35
324,Otjimbingwe,NaN,-22.35,16.13
372,Berbera,SO,10.44,45.01
374,Yulara,AU,-25.24,130.99
379,Bethanien,NaN,-26.50,17.16
382,Najran,SA,17.49,44.13


In [17]:
#loop thru data frame to collect hotels within the 5000 meter radius of the narrowed down weather criteria

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}
    
# taken from Airport_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params).json()
    
    print(f"Retrieving Results for: {row['City']}.")
    try:
        hotel_df.loc[index, 'Hotel Name'] = name_address['results'][0]['name']
    except Exception as e:
        print("Hotel not found..skipping")

Retrieving Results for: Taoudenni.
Hotel not found..skipping
Retrieving Results for: Port Hedland.
Retrieving Results for: Srivardhan.
Retrieving Results for: Marsh Harbour.
Retrieving Results for: Kibala.
Retrieving Results for: Otjimbingwe.
Hotel not found..skipping
Retrieving Results for: Berbera.
Retrieving Results for: Yulara.
Retrieving Results for: Bethanien.
Retrieving Results for: Najran.
Hotel not found..skipping
Retrieving Results for: Kununurra.
Retrieving Results for: Comodoro Rivadavia.
Retrieving Results for: Tagusao.
Hotel not found..skipping
Retrieving Results for: Buluang.
Hotel not found..skipping


In [12]:
hotel_df.head(20)

,City,Country,Latitude,Longitude,Hotel Name
11,Taoudenni,ML,22.68,-3.98,NaN
94,Port Hedland,AU,-20.32,118.57,The Esplanade Hotel
159,Srivardhan,IN,18.03,73.02,Tranquil Beach Resort
201,Marsh Harbour,BS,26.54,-77.06,Abaco Beach Resort
316,Kibala,AO,9.11,18.35,Hôtel des Chasses
324,Otjimbingwe,NaN,-22.35,16.13,NaN
372,Berbera,SO,10.44,45.01,Damal Hotel
374,Yulara,AU,-25.24,130.99,Sails in the Desert
379,Bethanien,NaN,-26.50,17.16,Bethanie Guesthouse
382,Najran,SA,17.49,44.13,NaN


In [18]:
#remove cities without hotels, thoses that have NaN values for hotel name
narrowed_city_df = hotel_df.dropna(how="any")
narrowed_city_df.head(20)

,City,Country,Latitude,Longitude,Hotel Name
94,Port Hedland,AU,-20.32,118.57,The Esplanade Hotel
159,Srivardhan,IN,18.03,73.02,Tranquil Beach Resort
201,Marsh Harbour,BS,26.54,-77.06,Abaco Beach Resort
316,Kibala,AO,9.11,18.35,Hôtel des Chasses
372,Berbera,SO,10.44,45.01,Damal Hotel
374,Yulara,AU,-25.24,130.99,Sails in the Desert
388,Kununurra,AU,-15.77,128.73,Kununurra Lakeside Resort
431,Comodoro Rivadavia,AR,-45.87,-67.50,Austral Hotel


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [20]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))

The hotel heat map shows the locations of hotels that meet the narrowed down weather criteria. Most of the hotels are near or below the equator.